<a href="https://colab.research.google.com/github/Benned-H/Summer2019/blob/master/Speech%20and%20Language%20Processing/3rd_Ed_Chap_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 2: RegEx, Text Normalization, Edit Distance

Given that my previous notes often become long-winded, I'm working to keep this down to definitions, explanations, and algorithms. Filler may be cut out.

## 2.1: Regular Expressions

**Regular expressions** (**RE**) are used to specify text search strings. **Text normalization** is the process of turning text into a more convenient form, which can include **tokenizing** words, or in the case of tweets or texts perhaps **emoticons** like ```:-)``` or **hashtags**. Languages like Chinese may not even have spaces between words, making tokenization more challenging. **Lemmatization** is the process of determining if two words share a root, e.g. *sing*, *sang*, and *sung*. **Stemming** is a simpler task of stripping suffixes off of words. Finally, text normalization can include **sentence segmentation**: breaking the text into sentences using punctuation cues. To compare words/strings, we'll introduce **edit distance** to measure how similar two strings are based on the number of edits between them.

We can specify **patterns** using RE to search a **corpus** of texts for all texts that match the pattern. Searches can be designed to return only first matches, full lines of documents, or entire documents. **Extended RE** gives easier syntax for patterns that simpler RE might need longer notation to specify. Here's the syntax:
* ```/str/``` specifies instances of the entered string, case sensitive.
* **Disjunction of characters**: ```/[abc]/``` specifies patterns containing either *a*, *b*, or *c*.
* **Range**: ```/[0-9]/``` is equivalent to ```/0123456789/```, ```/[3-7]/``` specifies *3*, *4*, *5*, *6*, or *7*, or ```/[A-Z]/``` specifies any uppercase letter.
* **Negation**: A caret after the bracket negates a disjunction, so ```/[^aA]/``` specifies any character expect *a* or *A*.
* Use ```/?/``` for "zero or one instance of the previous character, e.g. ```/boys?/``` finds *boy* or *boys*
* **Kleene \***: Means "zero or more instances of previous character or RE", e.g. an integer is ```/[0-9][0-9]*/```, as we need at least one digit.
* **Kleene +**: Means "one or more of previous character or RE", e.g. an integer is ```/[0-9]+/```.
* **Wildcard**: ```/./``` matches any single character, except a carriage return. Use a backslash to specify a regular period. Consider ```/.*/```.
* **Anchors** are special characters that require certain places in a string.
>a. The caret ```/^/``` matches the start of a line, so ```/^The/``` will only match *The* when it begins its line.   
>b. The dollar sign ```/$/``` matches the end of a line.   
>c. ```\b``` matches a word boundary. "Word" here means a sequence of digits, underscores, or letters. Hence ```/\bthe\b/``` will match *the* but not *other*.   
>d. ```\B``` matches a non-boundary.
* **Disjunction**: The pipe symbol means "or", e.g. ```/cat|dog/``` specifying *cat* or *dog*.

Clearly sequences take precedence over disjunction, but we can use parentheses to edit this behavior. Enclosing a RE in parentheses makes it act like a single character for surrounding operators. Thus ```/bab(y|ies)/``` specifies either the singular or plural. This enables the use of the Kleene star on expressions. We can formalize this precedence with a **operator precedence hierarchy**, showing which operators apply first:
1. Parenthesis: ```()```
2. Counters: ```* + ? {}```
3. Sequences/anchors: ```the ^my end$```
4. Disjunction: ```|```

Note that RE match the largest expression they can: they're **greedy**. We can specify **non-greedy** matching using the ```*?``` operator, a Kleene star that matches as little text as possible. ```+?``` is the same for a Kleene plus. As we use these operators to find desired patterns, we want to fix two kinds of errors: **false positives** and **false negatives**. Increasing **precision** means minimizing false positives, and increasing **recall** means minimizing false negatives.

There are a few more RE operators:
* **Curly brackets** can be used to specify given numbers of REs:   
>a. ```/{2}/``` matches exactly 2 occurrences of previous RE.   
>b. ```/{n,m}/``` matches from *n* to *m* occurrences of previous RE.   
>c. ```/{n,}/``` matches at least *n* occurrences of previous RE.   
>d. ```/{,m}/``` matches up to *m* occurrences of previous RE.
* ```\d``` matches any digit; it's equivalent to ```[0-9]```.
* ```\D``` matches any non-digit; equivalent to ```[^0-9]```.
* ```\w``` matches any alphanumeric or underscore.
* ```\W``` matches a non-alphanumeric or underscore; ```[^\w]```.
* ```\s``` matches any whitespace (space, tab).
* ```\S``` matches non-whitespace; ```[^\s]```.
* **Backslash**: Newline is ```\n```, tab is ```\t```. An asterisk, period, bracket, or question mark can all be referred to using a backslash. 

**Substitution** is an important use of RE. The substitution operator ```s/regexp1/pattern/``` replaces strings characterized by the RE to the other string. We can refer to string we find using parentheses on the first operator, and then use the **number** operator ```\1``` to refer back to it. For example:   
```s/([0-9]+)/<\1>/```. This puts angle brackets around all integers. We can specify that a certain string must occur twice in a text by using ```\1``` in the place of the second instance. The use of parentheses to stores these patterns is called a **capture group**. Each time parentheses surround a pattern, the match is stored in a numbered **register**. Thus we can use ```\2``` and then higher numbers for additional capture groups. Using parentheses for only grouping is called a **non-capturing group**, which is specified by putting ```?:``` after the open paren, in the form ```(?:pattern)```.

**Lookahead** assertions can be used to check for some pattern without advancing the cursor. The operator ```(?=pattern)``` is true if the pattern occurs, but it's **zero-width**, i.e. it doesn't advance the match pointer. Similarly, ```(?!pattern)``` is true only if the pattern *does not* occur, but it doesn't advance the cursor.

## 2.2: Words

A **corpus** is a computer-readable collection of text or speech. As we parse text from some corpus, whether we treat punctuation marks as their own words will depend on our task. In spoken language, an **utterance** is the spoken correlate of a sentence. **Disfluencies** are abnormalities that pop up in spoken speech, like **fragments** (broken off words) or **fillers** (words like *uh*, *um*). For text, should we differentiate between capitalized words and their lowercase versions?

A **lemma** is a set of lexical forms with the same stem, same part-of-speech, and same word sense. The **wordform** is the full inflected or derived form of the word. In languages such as Arabic, we'll often need to deal with lemmatization. **Types** are the number of distinct words in a corpus; vocabulary size $|V|$ equals the number of types. **Tokens** are the number $N$ of total running words in the corpus. Some popular corpora include:   
>Corpus | Tokens | Types
>:--- | ---: | ---:
>Shakespeare | 884k | 31k
>Brown corpus | 1mil | 38k
>Switchboard telephone conversations | 2.4mil | 20k
>COCA | 440mil | 2mil
>Google N-grams | 1tril | 13mil

The relationship between the number of types and number of tokens is called **Herdan's Law**, or **Heaps' Law** (In linguistics and information retrieval respectively). It's written as:   
$|V|=kN^\beta$, where $k$ and $\beta$ are positive constants and $0<\beta<1$. The value of $\beta$ depends on the corpus and genre, but generally the vocabulary size for a text goes up significantly faster than the square root of its number of tokens. Dictionaries can also give an approximate upper bound on the number of lemmas in a language.

## 2.3: Corpora

Perhaps the most important dimension of variation is the language; NLP algorithms are most useful when they generalize to other languages. This is also necessary for different dialects, such as African American Vernacular English (**AAVE**), a dialect spoken by millions in the US. Also common is **code switching**, where speakers or writers use multiple languages in a single communicative act. Corpora can also vary by genre, demographic, and historical period.

## 2.4: Text Normalization

At least three tasks are typical for a normalization process:
1. Segmenting/tokenizing words from running text.
2. Normalizing word formats.
3. Segmenting sentences from running text.

**Example in Unix commandline** - We'll use ```tr``` to systematically change characters in the input, ``sort`` to sort input lines in alphabetical order, and ``uniq``, which collapses and counts adjecent identical lines.

1. Use ```tr``` to translate characters. The format ```tr -tags str1 str2``` copies the standard input to the standard output with the characters in ```str1``` substituted to ```str2```. The ```-c``` tag *complements* the values in ```str1``` and the ```-s``` tag *squeezes* all sequences of ```str2``` in the output to a single instance.   
>Thus ```tr -sc "A-Za-z" "\n" < shakespeare.txt``` splits the corpus into individual alphabetic words on each line. We use the ```<``` to pass the text as input.
2. The ```sort``` utility sorts text by line, and we use a pipe ```|``` to take the input of ```tr``` and pass it to the next utility.
3. We then use ```uniq``` to read adjecent lines of the input and writes copies to the output, except duplicate lines aren't written. The ```-c``` tag precedes each line with a count of its occurances.   
>This gives us ```tr -sc "A-Za-z" "\n" < shakespeare.txt | sort | uniq -c``` so far.
4. We could also use ```tr A-Z a-z``` to convert all text to lowercase before sorting it.
5. Finally, we can use the ```sort``` tags ```-n``` to consider lines numerically and ```-r``` to sort in reverse order, giving us the largest values first. I also put this into a file to save our results. Our final command is:   
>```tr -sc "A-Za-z" "\n" < shakespeare.txt | tr A-Z a-z | sort | uniq -c | sort -n -r > word_counts.txt```

We'll need more advanced tools for **tokenization**, the task of segmenting running text into words, and **normalization**, the task of putting words/tokens into a standard format. We'll typically want to keep punctuation as tokens, but keep internal punctuation in words intact (e.g. *Ph.D.*, *$45.55*, *03/16/1999*, or *#nlproc*). We can also expand **clitic** contractions marked by apostrophes. Clitics are parts of words that can't stand on their own. We might also need to consider multiword expressions, which links this task to **named entity detection**, the task of detecting names, dates, and organizations.

A common tokenization standard is the **Penn Treebank tokenization** standard, used for parsed corpora released by the Linguistic Data Consortium (LDC). This separates clitics, keeps hyphenated words together, and separates out all punctuation. Tokens can be **normalized**, where a single form is chosen for words with multiple forms, e.g. *US* and *USA*. **Case folding** normalizes to lowercase, which might be desirable in speech recognition or IR, but maybe not sentiment analysis, IE, or MT. In practice, tokenization needs to be fast and deterministic, as it needs to happen before all other language processing.

In languages such as written Chinese, Japanese, and Thai, there are no spaces to mark word boundaries. Chinese words are composed of **hanzi** characters which are generally pronounced as a single morpheme. A simple algorithm called **maximum matching** (**MaxMatch**) performs quite well for segmenting Chinese, although it requires a wordlist of the language. It starts pointing at the beginning of the string and chooses the longest word in the dictionary currently matching the input. The pointer is advanced to the end of that string and the process repeats. If no word matches, advance the pointer a character (this will be a one-character word) and re-apply again. My version of the pseudocode:

In [0]:
def maxMatch(text,dictionary):
  # Returns a list of words in the sentence.
  if len(text) == 0:
    return []
  length = len(text)
  while length != 0: # All possible lengths will be checked.
    first_word = text[0:length]
    remainder = text[length:len(text)] # Rest of sentence.
    if first_word in dictionary:
      # Concatenate the found word with rest of sentence's result.
      return [first_word] + maxMatch(remainder,dictionary)
    length = length - 1

  # No word was found, so create a one-character word.
  first_word = text[0]
  remainder = text[1:]
  return [first_word] + maxMatch(remainder,dictionary)

This doesn't work as well on English, because Chinese has much shorter words. We can quantify how well the segmenter works using a metric called **word error rate**, where we compare our result with a perfect hand-segmented (i.e. **gold**) sentence. The WER is the normalized minimum edit distance between the two: the number of word insertions, deletions, and substitutions divided by the length of the gold sentence in words. We'll see how to calculate this soon.